<a href="https://colab.research.google.com/github/martinpius/Computer-Vission/blob/main/Model_subclassing_Inception_V3_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import tensorflow as tf
  print(f"You are using CoLab with tensorflow version: {tf.__version__}")
except Exception as e:
  print(f"{type(e)}: {e}\n...please load your drive...")
  COLAB = False
def time_fmt(x:float = 123.8918)->float:
  h = int(x /(60 * 60))
  m = int(x % (60 * 60) / 60)
  s = int(x % 60)
  return f"{h}: {m:>02}: {s:>05.2f}"
print(f"...time testing...time testing...time testing...\n>>>time elapse: {time_fmt()}")

Mounted at /content/drive
You are using CoLab with tensorflow version: 2.4.1
...time testing...time testing...time testing...
>>>time elapse: 0: 02: 03.00


In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
import time, os
import numpy as np

In [ ]:
#Convolutional block using layer subclassing:

In [ ]:
class ConvBlock(tf.keras.layers.Layer):
  def __init__(self, num_filters, kernels = 3, *args, **kwargs):
    super(ConvBlock, self).__init__(*args, **kwargs)
    self.conv = tf.keras.layers.Conv2D(filters = num_filters, kernel_size = kernels,
                                       kernel_initializer = 'random_normal', padding = 'same')
    self.bn = tf.keras.layers.BatchNormalization()
    self.act = tf.keras.layers.LeakyReLU(alpha = 0.3)

  def call(self, inputs_tensor, training = False):
    x = self.conv(inputs_tensor, training = training)
    x = self.act(x)
    x = self.bn(x)
    return x

In [ ]:
#Inception Block using layer subclassing:
class InceptionBlock(tf.keras.layers.Layer):
  def __init__(self, num_filters, *args, **kwargs):
    super(InceptionBlock, self).__init__(*args, **kwargs)
    self.block1 = ConvBlock(num_filters = num_filters[0])
    self.block2 = ConvBlock(num_filters = num_filters[1])
    self.block3 = ConvBlock(num_filters = num_filters[2])
    self.block4 = ConvBlock(num_filters = num_filters[3])
    self.max_pool = tf.keras.layers.MaxPooling2D()
    self.id_mapping = tf.keras.layers.Conv2D(filters = num_filters[1], kernel_size = 3, padding = 'same',activation = 'relu')

  def call(self, inputs_tensor, training = False):
    x = self.block1(inputs_tensor, training = training)
    x = self.block2(x, training = training)
    x = self.block3(x, training = training)
    x = self.block4(x + self.id_mapping(inputs_tensor), training = training)
    x = self.max_pool(x)
    return x

In [ ]:
#The model class using model subclassing:

In [ ]:
class ModelBlock(tf.keras.models.Model):
  def __init__(self, num_classes = 10, *args,**kwargs):
    super(ModelBlock, self).__init__(*args, **kwargs)
    self.resblock1 = InceptionBlock(num_filters = [32,64,64,128])
    self.resblock2 = InceptionBlock(num_filters = [64,128,128,256])
    self.resblock3 = InceptionBlock(num_filters = [128,256,256,512])
    self.glb = tf.keras.layers.GlobalAveragePooling2D()
    self.dense1 = tf.keras.layers.Dense(units = 1024, activation = 'relu', kernel_initializer = 'random_normal')
    self.drp = tf.keras.layers.Dropout(rate = 0.25)
    self.dense2 = tf.keras.layers.Dense(units = 512, activation = 'relu', kernel_initializer = 'random_normal')
    self.out = tf.keras.layers.Dense(units = 10, activation = 'softmax')
  
  def call(self, inputs_tensor, training = False):
    x = self.resblock1(inputs_tensor, training = training)
    x = self.resblock2(x, training = training)
    x = self.resblock3(x, training = training)
    x = self.glb(x)
    x = self.dense1(x, training = training)
    x = self.drp(x)
    x = self.dense2(x, training = training)
    x = self.out(x, training = training)
    return x

In [ ]:
#Instatiating the model class and training the model on cifar10 dataset:

In [ ]:
model = ModelBlock() 

In [ ]:
#Loading and preprocess cifar10 dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_rows, img_cols = 32, 32
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train, y_test = tf.keras.utils.to_categorical(y_train, num_classes = 10), tf.keras.utils.to_categorical(y_test, num_classes = 10)

In [ ]:
print(f"x_train_shape: {x_train.shape}, x_test_shape: {x_test.shape}\ny_train_shape: {y_train.shape}, y_test_shape: {y_test.shape}")

x_train_shape: (50000, 32, 32, 3), x_test_shape: (10000, 32, 32, 3)
y_train_shape: (50000, 10), y_test_shape: (10000, 10)


In [ ]:
#Converting into tensorflow datatype:

In [ ]:
BUFFER = len(x_train)
BATCH_SIZE = 64
EPOCHS = 30
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(BUFFER).batch(batch_size = BATCH_SIZE, drop_remainder = True)
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_data = test_data.batch(batch_size = BATCH_SIZE, drop_remainder = True)
x_train_sample_batch, y_train_sample_batch = next(iter(train_data))
print(f"x_train_batch_shape: {x_train_sample_batch.shape}, y_train_batch_shape: {y_train_sample_batch.shape}")

x_train_batch_shape: (64, 32, 32, 3), y_train_batch_shape: (64, 10)


In [ ]:
#The training loop:

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3)
loss_obj = tf.keras.losses.CategoricalCrossentropy()
train_metric = tf.keras.metrics.CategoricalAccuracy()
eval_metric = tf.keras.metrics.CategoricalAccuracy()
tic = time.time()
for epoch in range(EPOCHS):
  print(f"....start of epoch: {epoch + 1}\n>>>>please wait....")
  for (step, (x_train_batch, y_train_batch)) in enumerate(train_data):
    with tf.GradientTape() as tape:
      preds = model(x_train_batch, training = True)
      train_loss = loss_obj(y_train_batch, preds)
    grads = tape.gradient(train_loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_metric.update_state(y_train_batch, preds)
    train_acc = train_metric.result()
    train_metric.reset_states()
    if step % 200 == 0:
      print(f"epoch: {epoch + 1}, train accuracy: {float(train_acc):.4f}")
      print(f"batch: {step}, train loss: {float(train_loss):.4f}")
  for (step, (x_val_batch, y_val_batch)) in enumerate(test_data):
    preds = model(x_val_batch, training = False)
    eval_loss = loss_obj(y_val_batch, preds)
    eval_metric.update_state(y_val_batch, preds)
    val_acc = eval_metric.result()
    eval_metric.reset_states()
    if step % 200 == 0:
      print(f"epoch: {epoch + 1}, validation accuracy: {float(val_acc):.4f}")
      print(f"batch: {step}, validation loss: {float(eval_loss):.4f}")
toc = time.time()
print(f"\n>>> time elaplse for training and evaluation is: {time_fmt(toc- tic)}")

....start of epoch: 1
>>>>please wait....
epoch: 1, train accuracy: 0.6875
batch: 0, train loss: 0.9282
epoch: 1, train accuracy: 0.6250
batch: 200, train loss: 0.9329
epoch: 1, train accuracy: 0.5625
batch: 400, train loss: 1.2059
epoch: 1, train accuracy: 0.7969
batch: 600, train loss: 0.6342
epoch: 1, validation accuracy: 0.6719
batch: 0, validation loss: 1.0611
....start of epoch: 2
>>>>please wait....
epoch: 2, train accuracy: 0.6875
batch: 0, train loss: 0.9048
epoch: 2, train accuracy: 0.7969
batch: 200, train loss: 0.6250
epoch: 2, train accuracy: 0.7344
batch: 400, train loss: 0.7291
epoch: 2, train accuracy: 0.8281
batch: 600, train loss: 0.4422
epoch: 2, validation accuracy: 0.7500
batch: 0, validation loss: 0.7284
....start of epoch: 3
>>>>please wait....
epoch: 3, train accuracy: 0.8125
batch: 0, train loss: 0.5096
epoch: 3, train accuracy: 0.8438
batch: 200, train loss: 0.4096
epoch: 3, train accuracy: 0.8906
batch: 400, train loss: 0.3837
epoch: 3, train accuracy: 0.8750